In [ ]:
import multiprocessing
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
import shap
import matplotlib.pyplot as plt

def XGBClassiferForSurvey (fileName, testRatio = 0.2):
    """predict whether the surveyee will be craving given the survey
    
    Args: 
        fileName (str): name of file to read in data from 
        testRatio (float) : ratio of the test data in the whole dataset
    
    """    
    
    # Read in the raw data 
    # We will not sample data here because it is unlikely that we will have over 100k of data
    df = pd.read_parquet(fileName, engine='auto')
    
    # Work with NaN values, then update df
    # We will figure out how to deal with NaN after EDA 
    df = df
    
    # Transform the y variable to 0 and 1(xgb cannot handle factors well)
    df['is_craving'] = df['is_craving'].apply(lambda x: 0 if x=='False' else 1)
    
    # Move is_craving to the last column (skip if label is aleady the last colum)
    new_cols = [col for col in df.columns if col != 'is_craving'] + ['is_craving']
    df = df[new_cols]
    feature_columns = new_cols[:-1]

    # Isolate the x and y variables
    y = df.iloc[:, -1].values
    X = df.iloc[:, :-1].values
    
    #split dataset into training and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = testRatio)
    
    # Create xgboost matrices
    Train = xgb.DMatrix(X_train, label = y_train, feature_names = feature_columns)
    Test = xgb.DMatrix(X_test, label = y_test, feature_names = feature_columns)
    
    # Tuning parameters: Round 1
    print("=====Parameter Tunning Round 1=====")
    
    tune_control = KFold(n_splits = 5, shuffle = True).split(X = X_train, y = y_train)
    tune_grid = {'learning_rate': [0.05, 0.3],
                 'max_depth': range(2, 9, 2),
                 'colsample_bytree': [0.5, 1],
                 'subsample': [1],
                 'min_child_weight': [1],
                 'gamma': [0], 
                 'random_state': [1502],
                 'n_estimators': range(200, 2000, 200),
                 'booster': ["gbtree"]}
    
    classifier = XGBClassifier(objective = "binary:logistic")
    
    # Cross Validation Assembly
    nJobs = multiprocessing.cpu_count()
    grid_search = GridSearchCV(estimator = classifier,
                               param_grid = tune_grid,
                               scoring = "roc_auc",
                               n_jobs = nJobs,
                               cv = tune_control,
                               verbose = 5)

    # Setting evaluation parameters
    evaluation_parameters = {"early_stopping_rounds": 100,
                             "eval_metric": "auc",
                             "eval_set": [(X_test, y_test)]}

    # Hyperparameter tuning and cross validation
    tune_model = grid_search.fit(X = X_train,
                                 y = y_train,
                                 **evaluation_parameters)
    grid_search.best_params_, grid_search.best_score_
    
    
    # Tuning parameters: Round 2
    print("=====Parameter Tunning Round 2=====")
    
    tune_control = KFold(n_splits = 5, shuffle = True).split(X = X_train, y = y_train)
    tune_grid2 = {'learning_rate': [grid_search.best_params_.get('learning_rate')],
                   'max_depth': [grid_search.best_params_.get('max_depth')],
                   'colsample_bytree': [grid_search.best_params_.get('colsample_bytree')],
                   'subsample': [0.9, 1],
                   'min_child_weight': range(1,5,1),
                   'gamma': [0, 0.1], 
                   'n_estimators': range(200, 2000, 200),
                   'booster': ["gbtree"]}                                                

    #Cross Validation Assembly
    grid_search2 = GridSearchCV(estimator = classifier,
                               param_grid = tune_grid2,
                                scoring = "roc_auc",
                                n_jobs = nJobs,
                                cv = tune_control,
                                verbose = 5)

    # Hyperparameter tuning and cross validation
    tune_model2 = grid_search2.fit(X = X_train,
                                 y = y_train,
                                 **evaluation_parameters)
    grid_search2.best_params_, grid_search2.best_score_
    
    # We can continue tunning the hyperparameters...
    grid_search_last = grid_search2
    
    # Final model
    print("=====Final Model=====")
    finalParameters = {'learning_rate': grid_search_last.best_params_.get('learning_rate'),
                       'max_depth': grid_search_last.best_params_.get('max_depth'),
                       'colsample_bytree': grid_search_last.best_params_.get('colsample_bytree'),
                       'subsample': grid_search_last.best_params_.get('subsample'),
                       'min_child_weight': grid_search_last.best_params_.get('min_child_weight'),
                       'gamma': grid_search_last.best_params_.get('gamma'), 
                       'eval_metric': "auc",
                       'objective': "binary:logistic"}
    
    finalModel = xgb.train(params = finalParameters,
                   dtrain = Train,
                   num_boost_round = 800,
                   evals = [(Test, "Yes")],
                   verbose_eval = 50)
    
    # Predictions on test dataset
    predictions = finalModel.predict(Test)
    predictions = np.where(predictions > 0.05, 1, 0)
    
    # Model Diagnostic
    print("=====Model Diagnostic=====")
    # Confusion Matrix
    cm = confusion_matrix(y_test, predictions)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(cm)
    plt.title('Confusion Matrix')
    fig.colorbar(cax)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()
    
    # Print report
    report = classification_report(y_test, predictions)
    print(report)
    
    # ROC Curve and AUC
    fpr, tpr, _ = roc_curve(y_test, predictions)
    roc_auc = auc(fpr, tpr)
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([-0.02, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve')
    plt.legend(loc="lower right")
    plt.show()
    
    # feature importances
    xgb.plot_importance(finalModel, max_num_features = 10)
    
    # Prepare and plot SHAP 
    # monkey patch
    model_bytearray = finalModel.save_raw()[4:]
    def myfun(self=None):
        return model_bytearray
    finalModel.save_raw = myfun
    # plot SHAP
    explainer = shap.TreeExplainer(finalModel)
    shap_values = explainer.shap_values(X_test)
    shap.summary_plot(shap_values,
                      X_test,
                      feature_names = feature_columns,
                      max_display = 10)


## NOTES

1. I am not sure if it is a time series problem. If it is a time series problem, we might split data (into training, validation and test) by the time point, and fine tune the hyperparameters differently. I used to fine tune the hyperparemeters using for loop, but I think there might be better ways..(?) 

2. for each features, shall we use "lag" to add new features like "happy_lag_1", "happy_lag_2", "stress_lag_1" etc. which will suggest the surveyee's mood at the last time(s) he took the survey. It might depend on the featrue importance plot(?) For example, if we know anxious is mostly related to is_craving, shall we add more lag of "anxious" in the features?

3. Do we need to add some more features to for more accurate predictions? Maybe some domain knowledge are required.

4. Further discussion on how to fill the NaNs and how to do the interpolation is needed. Maybe after the primary EDA on the real example. 

5. Is there any value we would like to return from the above function?